<a href="https://colab.research.google.com/github/stwater20/AIS3-2024-Material/blob/main/AIS3_Lab1_threat_entity_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#安裝必要套件~~耐心等待喔
!pip3 install datasets seqeval evaluate huggingface transformers accelerate>=0.21.0

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Sequence, Features
import pandas as pd

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# 整理資料集
def load_data(file_path):
  tokens = []
  ner_tags = []
  temp_tokens = []
  temp_ner_tags = []
  with open(file_path, 'r', encoding='utf-8') as file:
      for line in file:
          if len(line.split())>1:
              line = line.strip()
              token, tag = line.rsplit(' ', 1)
              temp_tokens.append(token)
              temp_ner_tags.append(tag)
          else:
              tokens.append(temp_tokens)
              ner_tags.append(temp_ner_tags)
              temp_tokens = []
              temp_ner_tags = []
  ds = Dataset.from_dict({'tokens': tokens, 'ner_tags': ner_tags})
  return ds

ds_train = load_data('DNRTI/train.txt')
ds_valid = load_data('DNRTI/valid.txt')
ds_test = load_data('DNRTI/test.txt')

In [ ]:
# 有時候我們不知道 Tag 有哪幾類, Set 的目的會取出唯一的值
unique_tags = set()
for dataset in [ds_train, ds_valid, ds_test]:
    unique_tags.update([tag for sublist in dataset['ner_tags'] for tag in sublist])
unique_tags = sorted(list(unique_tags))
# unique_tags

In [ ]:
# 模型訓練時是告訴模型對應的數字是多少，這可以減少記憶體使用量, 當然之後為了要給人看需要還原回去
label2id = {label: idx for idx, label in enumerate(unique_tags)}
id2label = {idx: label for label, idx in label2id.items()}

In [ ]:
def update_ner_tags(example):
    # 更新NER標籤：將標籤轉換為相應的ID
    return {'ner_tags': [label2id[tag] for tag in example['ner_tags']]}

# 使用update_ner_tags函數映射數據集中的每個樣本，將標籤轉為ID
# map是將傳入的函數應用於可迭代對象的每個元素上，並返回一個新的迭代器，其中包含了應用函數後的結果。
ds_train = ds_train.map(update_ner_tags)
ds_valid = ds_valid.map(update_ner_tags)
ds_test = ds_test.map(update_ner_tags)

Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [ ]:
# DatasetDict 可以方便之後的操作 (應用變換)
# https://huggingface.co/docs/datasets/v2.20.0/en/package_reference/main_classes#datasets.DatasetDict
raw_datasets = DatasetDict({
    'train': ds_train,
    'valid': ds_valid,
    'test': ds_test
})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5439
    })
    valid: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 668
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 679
    })
})

In [ ]:
# 印出資料集的 Sample
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = id2label[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

The admin@338 has largely targeted organizations involved in financial , economic and trade  policy , typically using publicly available RATs   such as Poison Ivy    , as well some non-public backdoors . 
O   B-HackOrg O   O       O        O             O        O  B-Idus    O B-Idus   O   B-Idus I-Idus O O         O     B-Tool   I-Tool    I-Tool O    O  B-Tool I-Tool O O  O    O    B-Tool     I-Tool    O 


In [ ]:
# 下載分詞器準備處理文字

from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
#分詞出的結果, 會以 ## 表示原本與前面的單詞合併是原始單詞
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()[:5]

['[CLS]', 'The', 'ad', '##min', '@']

In [ ]:
# 將標籤（labels）與分詞後的tokens對齊
# 原始數據集中的標籤通常是針對整個詞的，但分詞器可能將詞拆分成多個子詞（subwords），所以需要重新對標籤進行對齊以保持一致性。
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[26, 3, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 26, 26, 11, 24, 26, 26, 26, 26, 11, 24, 26]
[-100, 26, 3, 4, 4, 4, 4, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 24, 24, 26, 26, 11, 12, 24, 26, 26, 26, 26, 11, 12, 12, 24, 24, 24, 26, -100]


In [ ]:
#為了預處理整個資料集，我們需要對所有輸入進行標記並應用於align_labels_with_tokens()所有標籤。
#利用快速分詞器同時對大量文字進行分詞，所以我寫了個處理範例清單的函數，並使用Dataset.map()帶有選項的方法batched=True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i) # 對於分詞結果中的每個樣本，利用tokenized_inputs.word_ids(i)獲取每個token對應的原始詞的索引。
        new_labels.append(align_labels_with_tokens(labels, word_ids)) # 將每個詞的NER標記對應到分詞後產生的tokens上, 加回新的 labels list

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True, #批量處理，一次處理多個樣本，提高效率
    remove_columns=raw_datasets["train"].column_names, # 移除原始數據集中的所有列，只保留處理後的數據
)

Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

# 創建一個數據整理器的實例，並將分詞器作為參數傳遞，能夠自動對短於最大序列長度的樣本進行填充，並將超過最大長度的樣本進行截斷
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,   26,    3,    4,    4,    4,    4,   26,   26,   26,   26,   26,
           26,    4,   26,    4,   26,    4,   17,   26,   26,   26,   11,   24,
           24,   24,   24,   26,   26,   11,   12,   24,   26,   26,   26,   26,
           11,   12,   12,   24,   24,   24,   26, -100],
        [-100,   26,    3,    4,    4,    4,    4,   26,   26,    0,   13,    6,
           19,   26,   26,   26,   26,   26,    4,   26,    4,   26,   26,    0,
           13,   26,    0,   26, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]])

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 26, 3, 4, 4, 4, 4, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 24, 24, 26, 26, 11, 12, 24, 26, 26, 26, 26, 11, 12, 12, 24, 24, 24, 26, -100]
[-100, 26, 3, 4, 4, 4, 4, 26, 26, 0, 13, 6, 19, 26, 26, 26, 26, 26, 4, 26, 4, 26, 26, 0, 13, 26, 0, 26, -100]


In [ ]:
import evaluate
# seqeval 是一個常用於評估序列標記任務的指標，如命名實體識別（NER）、詞性標注（POS tagging）等。它可以計算多個常用的評估指標，包括精確度（Precision）、召回率（Recall）和 F1 分數。
metric = evaluate.load("seqeval")

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [id2label[i] for i in labels]
labels

['O',
 'B-HackOrg',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Idus',
 'O',
 'B-Idus',
 'O',
 'B-Idus',
 'I-Idus',
 'O',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'I-Tool',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'O',
 'O',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'O']

In [ ]:
predictions = labels.copy()
predictions[1] = "O"
metric.compute(predictions=[predictions], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'HackOrg': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'Idus': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'Tool': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 0.8571428571428571,
 'overall_f1': 0.923076923076923,
 'overall_accuracy': 0.967741935483871}

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 使用 argmax 獲得最有可能的標籤索引

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

27

In [ ]:
from transformers import TrainingArguments
# 初始化訓練參數
args = TrainingArguments(
    "bert-finetuned-ner",          # 訓練過程中保存模型和結果的目錄名稱
    evaluation_strategy="epoch",   # 設定評估策略為每一個epoch後進行一次評估
    save_strategy="epoch",         # 設定保存策略為每一個epoch後保存模型
    learning_rate=2e-5,            # 設定學習率
    num_train_epochs=3,            # 設定訓練的總epoch數
    weight_decay=0.01,             # 設定權重衰減，用於避免過擬合
    push_to_hub=True,              # 設定是否將訓練好的模型推送到 Hugging Face 的 Model Hub
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.775200,0.459374,0.689949,0.716012,0.702739,0.859694
2,0.419200,0.412117,0.729357,0.774034,0.751032,0.877593
3,0.263000,0.405540,0.737449,0.794793,0.765048,0.885831


TrainOutput(global_step=2040, training_loss=0.441149780329536, metrics={'train_runtime': 366.768, 'train_samples_per_second': 44.489, 'train_steps_per_second': 5.562, 'total_flos': 489718522873548.0, 'train_loss': 0.441149780329536, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/ShengShan/bert-finetuned-ner/commit/6e977d014df2d567fdffd606522398361cb9164c', commit_message='Training complete', commit_description='', oid='6e977d014df2d567fdffd606522398361cb9164c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.29763174057006836,
 'eval_precision': 0.7847121432027092,
 'eval_recall': 0.8342190982341848,
 'eval_f1': 0.8087086588000665,
 'eval_accuracy': 0.9119802317655078,
 'eval_runtime': 2.8344,
 'eval_samples_per_second': 239.554,
 'eval_steps_per_second': 29.988,
 'epoch': 3.0}

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "ShengShan/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier(["Eset‍ has published a report on the state-sponsored Russian turla apt group."])

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

[[{'entity_group': 'SecTeam',
   'score': 0.9920218,
   'word': 'E',
   'start': 0,
   'end': 1},
  {'entity_group': 'Time',
   'score': 0.9950605,
   'word': '##set',
   'start': 1,
   'end': 4},
  {'entity_group': 'Area',
   'score': 0.9228633,
   'word': 'Russian',
   'start': 52,
   'end': 59},
  {'entity_group': 'HackOrg',
   'score': 0.60031486,
   'word': 't',
   'start': 60,
   'end': 61},
  {'entity_group': 'Idus',
   'score': 0.38845345,
   'word': '##la',
   'start': 63,
   'end': 65},
  {'entity_group': 'HackOrg',
   'score': 0.72458035,
   'word': 'apt group',
   'start': 66,
   'end': 75}]]